# Рассчитать данные по энергопотреблению

## Описание задания
Загрузите данные и посчитайте модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей: в первой модели весь оптимальный набор метеорологических данных, во второй - дни недели и праздники, в третьей - недели года, в четвертой - месяцы. Финальное значение показателя рассчитайте как взвешенное арифметическое показателей всех моделей, взяв веса для первой и второй модели как 3/8, а для третьей и четвертой - как 1/8.

Загрузите данные решения, посчитайте значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузите результат в виде CSV-файла (submission.csv).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from scipy.interpolate import interp1d

### Вспомогательные функции

In [ ]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

## Данные для обучения

In [ ]:
# Загрузка
buildings = pd.read_csv('./data/building_metadata.csv.gz', usecols=['site_id', 'building_id'])

weather = pd.read_csv('./data/weather_train.csv.gz')
# Возьмем только site_id = 0 для сокращения времени расчета
weather = weather[weather['site_id'] == 0]

energy = pd.read_csv('./data/train.0.csv.gz')
# Берем данные по энергопотреблению для первых 50 зданий
energy = energy[energy['building_id'] < 50]
energy = energy[energy["meter_reading"] > 0]

In [ ]:
# Данные погоды
## Интерполяция пропусков
weather["precip_depth_1_hr"] = weather["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed", "wind_direction",
                       "precip_depth_1_hr", "sea_level_pressure"]
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic')

## Обогащение данных
weather["wind_direction_rad"] = weather["wind_direction"] * np.pi/180
weather["wind_direction_sin"] = np.sin(weather["wind_direction_rad"])
weather["wind_direction_cos"] = np.cos(weather["wind_direction_rad"])
weather["air_temperature_diff1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]

In [ ]:
# Данные по энергопотреблению
## Обогащение данных
energy['timestamp'] = pd.to_datetime(energy['timestamp'])
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
energy["week"] = energy["timestamp"].dt.week.astype("int8")
energy["month"] = energy["timestamp"].dt.month.astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
for week in range(1,54):
    energy['is_w' + str(week)] = energy['week'].isin([week]).astype("int8")
for month in range(1,13):
    energy['is_m' + str(month)] = energy['month'].isin([month]).astype("int8")

energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

In [ ]:
#  Объединим данные
energy = pd.merge(left=energy, right=buildings, how='left', left_on='building_id', right_on='building_id')

energy = energy.set_index(["timestamp", "site_id"])
weather['timestamp'] = pd.to_datetime(weather['timestamp'])
weather = weather.set_index(["timestamp", "site_id"])
data_train = pd.merge(left=energy, right=weather, how="left", left_index=True, right_index=True)
data_train.reset_index(inplace=True)

In [ ]:
#  Удалим ненужные столбцы и оптимизируем памать
data_train = data_train.drop(columns=['meter', 'site_id', 'timestamp'], axis=1)
data_train = reduce_mem_usage(data_train)
del weather
del energy
print (data_train.info())

## Строим модели

In [ ]:
# Набор параметров для каждой модели
## Модель 1. Весь оптимальный набор метео данных
lr_weather_columns = ["meter_reading_log", "hour", "building_id",
                      "air_temperature", "dew_temperature",
                      "sea_level_pressure", "wind_speed",
                      "air_temperature_diff1", "air_temperature_diff2",
                      "cloud_coverage",
                      "wind_direction_rad", "wind_direction_sin"]

## Модель 2. Дни недели и праздники                      
lr_day_columns = ["meter_reading_log", "hour", "building_id", "is_holiday"]
for wday in range(0,7):
    lr_day_columns.append("is_wday" + str(wday))

## Модель 3. Недели года                      
lr_week_columns = ["meter_reading_log", "hour", "building_id"]
for week in range(1,54):
    lr_week_columns.append("is_w" + str(week))

## Модель 4. Месяцы                      
lr_month_columns = ["meter_reading_log", "hour", "building_id"]
for month in range(1,13):
    lr_month_columns.append("is_m" + str(month))

hours = range(0, 24)
building_ids = range(0, data_train["building_id"].max() + 1)

In [ ]:
def train_model (df, columns):
    df_train_lr = pd.DataFrame(df, columns=columns)
    lr_models = [[]]*len(buildings)
    for building in building_ids:
        lr_models[building] = [[]]*len(hours)
        df_train_b = df_train_lr[df_train_lr["building_id"]==building]
        for hour in hours:
            df_train_bh = df_train_b[df_train_b["hour"]==hour]
            y = df_train_bh["meter_reading_log"]
            x = df_train_bh.drop(labels=["meter_reading_log", "hour", "building_id"], axis=1)
            model = LinearRegression(fit_intercept=False).fit(x, y)
            lr_models[building][hour] = model.coef_
            lr_models[building][hour] = np.append(lr_models[building][hour], model.intercept_)
    return lr_models

In [ ]:
# Расчет отдельных моделей
lr_weather_models = train_model(data_train, lr_weather_columns)
lr_day_models = train_model(data_train, lr_day_columns)
lr_week_models = train_model(data_train, lr_week_columns)
lr_month_models = train_model(data_train, lr_month_columns)

In [ ]:
def calculate_model_ensemble (x, model, columns):
    lr = -1
    if len(model) > 0:
        lr = np.sum([x[col] * model[i] for i,col in enumerate(columns[3:])])
        # lr += model[len(columns)-3]
        lr += model[-1]
        lr = np.exp(lr)
    if lr < 0 or lr*lr == lr:
        lr = 0

    return lr

def calculate_models_ensemble (x):
    lr_weather = calculate_model_ensemble(x, model=lr_weather_models[x.building_id][x.hour], columns=lr_weather_columns)
    lr_day = calculate_model_ensemble(x, model=lr_day_models[x.building_id][x.hour], columns=lr_day_columns)
    lr_week = calculate_model_ensemble(x, model=lr_week_models[x.building_id][x.hour], columns=lr_week_columns)
    lr_month = calculate_model_ensemble(x, model=lr_month_models[x.building_id][x.hour], columns=lr_month_columns)    
    lr_ensemble = lr_weather * 3/8 + lr_day * 3/8 + lr_week * 1/8 + lr_month * 1/8

    x['meter_reading'] = lr_ensemble

    return x

## Данные для тестирования

In [ ]:
# Загрузка
weather = pd.read_csv('./data/weather_test.csv.gz')
# Возьмем только site_id = 0 для сокращения времени расчета
weather = weather[weather['site_id'] == 0]

energy = pd.read_csv('./data/test.csv.gz')
# Берем данные по энергопотреблению для первых 50 зданий
energy = energy[energy['building_id'] < 50]
print(energy.info())

In [ ]:
# Данные погоды
## Интерполяция пропусков
weather["precip_depth_1_hr"] = weather["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed", "wind_direction",
                       "precip_depth_1_hr", "sea_level_pressure"]
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic')

## Обогащение данных
weather["wind_direction_rad"] = weather["wind_direction"] * np.pi/180
weather["wind_direction_sin"] = np.sin(weather["wind_direction_rad"])
weather["wind_direction_cos"] = np.cos(weather["wind_direction_rad"])
weather["air_temperature_diff1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]

In [ ]:
# Данные по энергопотреблению
## Обогащение данных
energy['timestamp'] = pd.to_datetime(energy['timestamp'])
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
energy["week"] = energy["timestamp"].dt.week.astype("int8")
energy["month"] = energy["timestamp"].dt.month.astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
for week in range(1,54):
    energy['is_w' + str(week)] = energy['week'].isin([week]).astype("int8")
for month in range(1,13):
    energy['is_m' + str(month)] = energy['month'].isin([month]).astype("int8")

In [ ]:
#  Объединим данные
energy = pd.merge(left=energy, right=buildings, how='left', left_on='building_id', right_on='building_id')

energy = energy.set_index(["timestamp", "site_id"])
weather['timestamp'] = pd.to_datetime(weather['timestamp'])
weather = weather.set_index(["timestamp", "site_id"])
data_test = pd.merge(left=energy, right=weather, how="left", left_index=True, right_index=True)
data_test.reset_index(inplace=True)

In [ ]:
#  Удалим ненужные столбцы и оптимизируем памать
data_test = data_test.drop(columns=['meter', 'site_id', 'timestamp'], axis=1)
data_test = reduce_mem_usage(data_test)
del weather
del energy
print (data_test.info())

In [ ]:
# Рассчитаем тестовые данные
data_test = data_test.apply(calculate_models_ensemble, axis=1, result_type='expand')

In [ ]:
# Подготовка результатов в требуемом формате
results_ready = pd.DataFrame(data_test, columns=["row_id", "meter_reading"])

results = pd.read_csv("./data/test.csv.gz", usecols=["row_id"])
results = pd.merge(left=results, right=results_ready, how="left", left_on="row_id", right_on="row_id")
results.fillna(value=0, inplace=True)
print (results.info())

In [ ]:
results.to_csv("./data/submission.csv",index=False)

In [ ]:
del data_train
del data_test
del results
del results_ready